# regress.ipynb
Author:  Kevin Tran <ktran@andrew.cmu.edu>

This python notebook performs regressions on data pulled from a processed mongo DB created by GASpy. It then saves these regressions into pickles (for later use) and creates parity plots of the regression fits.

## Initializations

###### Imports

In [1]:
import pdb
from pprint import pprint
import copy
import sys
import math
import numpy as np
from pymatgen.matproj.rest import MPRester
from sklearn import metrics
import dill as pickle
pickle.settings['recurse'] = True     # required to pickle lambdify functions
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.plotly as py
import plotly.graph_objs as go

###### Load data

In [2]:
sys.path.append('..')
from gaspy.utils import vasp_settings_to_str
from gas_pull import GASPull

# Calculation settings we want to look at
#VASP_SETTINGS = vasp_settings_to_str({'gga': 'RP',
#                                      'pp_version': '5.4.',
#                                      'encut': 350})
VASP_SETTINGS = vasp_settings_to_str({})

# Pull the data from the database. We do it once for each set of features, since each
# set of features will create a different shape of `X`
GAS_PULL = GASPull(VASP_SETTINGS, split=True)
FEATURE_SETS = [
                'energy_fr_coordcount',
                'energy_fr_coordcount_ads',
                'energy_fr_coordcount_nncoord_ads',
                #'energy_fr_gcn_ads'
                ]
X = dict.fromkeys(FEATURE_SETS)
Y = dict.fromkeys(FEATURE_SETS)
DATA = dict.fromkeys(FEATURE_SETS)
X_TRAIN = dict.fromkeys(FEATURE_SETS)
X_TEST = dict.fromkeys(FEATURE_SETS)
Y_TRAIN = dict.fromkeys(FEATURE_SETS)
Y_TEST = dict.fromkeys(FEATURE_SETS)
lb_ads = dict.fromkeys(FEATURE_SETS)
lb_coord = dict.fromkeys(FEATURE_SETS)
# Some of the methods we use have more outputs than others. We EAFP to address that.
for feature_set in FEATURE_SETS:
    try:
        X[feature_set], Y[feature_set], DATA[feature_set], \
        X_TRAIN[feature_set], X_TEST[feature_set], \
        Y_TRAIN[feature_set], Y_TEST[feature_set], \
        lb_ads[feature_set], lb_coord[feature_set] = \
                getattr(GAS_PULL, feature_set)()
    except ValueError:
        X[feature_set], Y[feature_set], DATA[feature_set], \
        X_TRAIN[feature_set], X_TEST[feature_set], \
        Y_TRAIN[feature_set], Y_TEST[feature_set], \
        lb_coord[feature_set] = \
                getattr(GAS_PULL, feature_set)()

# Do some prep-work for non-ads models
ADS = np.unique(DATA[DATA.keys()[0]]['adsorbate'])

## Regressions
Create surrogate models using different methods

###### SKLearn Gaussian Process Regressor

In [12]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels \
    import RBF, WhiteKernel, RationalQuadratic, ExpSineSquared

# Specify the kernel to use. If it's `None`, then it uses SKLearn's default RBF
#K = 1.0*RBF(length_scale=1.0) + 1.0*WhiteKernel(noise_level=0.05**2.0) 
K = None

# Initialize
GP = dict.fromkeys(FEATURE_SETS)
GP_TEMPLATE = GaussianProcessRegressor(kernel=K, n_restarts_optimizer=0)

# Perform the regression for all of our feature sets
for feature_set in FEATURE_SETS:
    GP[feature_set] = {}
    GP[feature_set]['model_name'] = 'GP'
    
    # Loop through `ADS` if the model does not have `ads` as a feature
    if not lb_ads[feature_set]:
        GP[feature_set]['model'] = {}
        for ads in ADS:
            x_train = np.array([x for i, x in enumerate(X_TRAIN[feature_set])
                                if DATA[feature_set]['adsorbate'][i] == ads])
            y_train = np.array([y for i, y in enumerate(Y_TRAIN[feature_set])
                                if DATA[feature_set]['adsorbate'][i] == ads])
            GP[feature_set]['model'][ads] = copy.deepcopy(GP_TEMPLATE)
            GP[feature_set]['model'][ads].fit(x_train, y_train)
            
    # Perform a normal regression if `ads` is a feature
    else:
        GP[feature_set]['model'] = copy.deepcopy(GP_TEMPLATE)
        GP[feature_set]['model'].fit(X_TRAIN[feature_set], Y_TRAIN[feature_set])

In [13]:
# Save the GP regressions
for feature_set in FEATURE_SETS:
    pkl = {'model': GP[feature_set],
           'pre_processors': {'coordination': lb_coord[feature_set],
                              'adsorbate': lb_ads[feature_set]}}
    with open('pkls/' + GP[feature_set]['model_name'] + '_' \
              + feature_set + '.pkl', 'wb') as fname:
        pickle.dump(pkl, fname)

In [3]:
# Open the GP models
GP = dict.fromkeys(FEATURE_SETS)
for feature_set in FEATURE_SETS:
    with open('pkls/GP_' + feature_set + '.pkl', 'r') as fname:
        pkl = pickle.load(fname)
    GP[feature_set] = pkl['model']

###### TPOT Regression

In [9]:
from tpot import TPOTRegressor

# Initialize
TPOT = dict.fromkeys(FEATURE_SETS)
TPOT_TEMPLATE = TPOTRegressor(generations=1,
                              population_size=2,
                              verbosity=2,
                              random_state=42)

# Perform the regression for all of our feature sets
for feature_set in FEATURE_SETS:
    TPOT[feature_set] = {}
    TPOT[feature_set]['model_name'] = 'TPOT'
    
    # Loop through `ADS` if the model does not have `ads` as a feature
    if not lb_ads[feature_set]:
        TPOT[feature_set]['model'] = {}
        for ads in ADS:
            x_train = np.array([x for i, x in enumerate(X_TRAIN[feature_set])
                                if DATA[feature_set]['adsorbate'][i] == ads])
            y_train = np.array([y for i, y in enumerate(Y_TRAIN[feature_set])
                                if DATA[feature_set]['adsorbate'][i] == ads])
            TPOT[feature_set]['model'][ads] = copy.deepcopy(TPOT_TEMPLATE)
            TPOT[feature_set]['model'][ads].fit(x_train, y_train)
            
    # Perform a normal regression if `ads` is a feature
    else:
        TPOT[feature_set]['model'] = copy.deepcopy(TPOT_TEMPLATE)
        TPOT[feature_set]['model'].fit(X_TRAIN[feature_set], Y_TRAIN[feature_set])

Generation 1 - Current best internal CV score: 0.971962197916

Best pipeline: RandomForestRegressor(input_matrix, RandomForestRegressor__bootstrap=True, RandomForestRegressor__max_features=0.75, RandomForestRegressor__min_samples_leaf=11, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=100)


Generation 1 - Current best internal CV score: 0.65849927395

Best pipeline: RandomForestRegressor(input_matrix, RandomForestRegressor__bootstrap=True, RandomForestRegressor__max_features=0.5, RandomForestRegressor__min_samples_leaf=11, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=100)


Generation 1 - Current best internal CV score: 0.744414764085

Best pipeline: RandomForestRegressor(input_matrix, RandomForestRegressor__bootstrap=True, RandomForestRegressor__max_features=0.5, RandomForestRegressor__min_samples_leaf=11, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=100)


Generation 1 - Current best internal CV score: 0.921267355157

Best pipeline: RandomForestRegressor(input_matrix, RandomForestRegressor__bootstrap=True, RandomForestRegressor__max_features=0.75, RandomForestRegressor__min_samples_leaf=11, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=100)


Generation 1 - Current best internal CV score: 1.21612071218

Best pipeline: RandomForestRegressor(input_matrix, RandomForestRegressor__bootstrap=True, RandomForestRegressor__max_features=0.5, RandomForestRegressor__min_samples_leaf=11, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=100)


Generation 1 - Current best internal CV score: 0.779903273224

Best pipeline: KNeighborsRegressor(ElasticNetCV(input_matrix, ElasticNetCV__l1_ratio=0.35, ElasticNetCV__tol=0.1), KNeighborsRegressor__n_neighbors=80, KNeighborsRegressor__p=1, KNeighborsRegressor__weights=distance)


Generation 1 - Current best internal CV score: 0.324042906056

Best pipeline: KNeighborsRegressor(ElasticNetCV(input_matrix, ElasticNetCV__l1_ratio=0.35, ElasticNetCV__tol=0.1), KNeighborsRegressor__n_neighbors=80, KNeighborsRegressor__p=1, KNeighborsRegressor__weights=distance)


Generation 1 - Current best internal CV score: 0.315339833974

Best pipeline: RandomForestRegressor(input_matrix, RandomForestRegressor__bootstrap=True, RandomForestRegressor__max_features=0.5, RandomForestRegressor__min_samples_leaf=11, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=100)


In [10]:
# Save the TPOT regressions
for feature_set in FEATURE_SETS:
    # Turn the TPOT objects into pipelines so that we can pickle them
    if not lb_ads[feature_set]:
        for ads in ADS:
            TPOT[feature_set]['model'][ads] = \
                TPOT[feature_set]['model'][ads].fitted_pipeline_
    else:
        TPOT[feature_set]['model'] = TPOT[feature_set]['model'].fitted_pipeline_
    
    pkl = {'model': TPOT[feature_set],
           'pre_processors': {'coordination': lb_coord[feature_set],
                              'adsorbate': lb_ads[feature_set]}}
    with open('pkls/' + TPOT[feature_set]['model_name'] + '_' \
              + feature_set + '.pkl', 'wb') as fname:
        pickle.dump(pkl, fname)

In [4]:
# Open the TPOT models
TPOT = dict.fromkeys(FEATURE_SETS)
for feature_set in FEATURE_SETS:
    with open('pkls/TPOT_' + feature_set + '.pkl', 'r') as fname:
        pkl = pickle.load(fname)
    TPOT[feature_set] = pkl['model']

###### Alamo Regression

In [8]:
import alamopy

# Initialize
ALAMO = dict.fromkeys(FEATURE_SETS)
def call_alamo(x_train, x_test, y_train, y_test):
    '''
    This function calls ALAMO given a set of training and test data. The settings
    are hard-coded.
    '''
    # Reshape the output because Alamo is like that
    y_train = y_train.reshape(len(y_train), 1)
    y_test = y_test.reshape(len(y_test), 1)
    # Call alamo
    ala_model = alamopy.doalamo(x_train, y_train, x_test, y_test,
                                showalm=1,
                                linfcns=1,
                                expfcns=1,
                                logfcns=1,
                                monomialpower=(1, 2, 3),
                                multi2power=(1, 2, 3),
                                ratiopower=(1, 2, 3))
    return ala_model

# Perform the regression for all of our feature sets
for feature_set in FEATURE_SETS:
    ALAMO[feature_set] = {}
    ALAMO[feature_set]['model_name'] = 'ALAMO'
    
    # Loop through `ADS` if the model does not have `ads` as a feature
    if not lb_ads[feature_set]:
        ALAMO[feature_set]['model'] = {}
        for ads in ADS:
            x_train = np.array([x for i, x in enumerate(X_TRAIN[feature_set])
                                if DATA[feature_set]['adsorbate'][i] == ads])
            x_test = np.array([x for i, x in enumerate(X_TEST[feature_set])
                                if DATA[feature_set]['adsorbate'][i] == ads])
            y_train = np.array([y for i, y in enumerate(Y_TRAIN[feature_set])
                                if DATA[feature_set]['adsorbate'][i] == ads])
            y_test = np.array([y for i, y in enumerate(Y_TEST[feature_set])
                                if DATA[feature_set]['adsorbate'][i] == ads])
            ALAMO[feature_set]['model'][ads] = call_alamo(x_train, x_test,
                                                          y_train, y_test)
            
    # Perform a normal regression if `ads` is a feature
    else:
        ALAMO[feature_set]['model'] = copy.deepcopy(ALAMO_TEMPLATE)
        ALAMO[feature_set]['model'].fit(X_TRAIN[feature_set], Y_TRAIN[feature_set])

Xmin and Xmax are not provided, they will be calculated from the training data
Alamo encountered an error, please investigate the .alm file


NameError: global name 'quit' is not defined

In [10]:
# Save the ALAMO regressions
for feature_set in FEATURE_SETS:
    # Turn the ALAMO objects into pipelines so that we can pickle them
    if not lb_ads[feature_set]:
        for ads in ADS:
            ALAMO[feature_set]['model'][ads] = \
                ALAMO[feature_set]['model'][ads].fitted_pipeline_
    else:
        ALAMO[feature_set]['model'] = ALAMO[feature_set]['model'].fitted_pipeline_
    
    pkl = {'model': ALAMO[feature_set],
           'pre_processors': {'coordination': lb_coord[feature_set],
                              'adsorbate': lb_ads[feature_set]}}
    with open('pkls/' + ALAMO[feature_set]['model_name'] + '_' \
              + feature_set + '.pkl', 'wb') as fname:
        pickle.dump(pkl, fname)

In [5]:
# Open the ALAMO models
ALAMO = dict.fromkeys(FEATURE_SETS)
for feature_set in FEATURE_SETS:
    with open('pkls/ALAMO_' + feature_set + '.pkl', 'r') as fname:
        pkl = pickle.load(fname)
    ALAMO[feature_set] = pkl['model']

## Plotting

###### SKLearn-types

In [5]:
# For each feature set and model combination
RMSE = dict.fromkeys(FEATURE_SETS)
for feature_set in FEATURE_SETS:
    # Pull out information/initialize
    data = DATA[feature_set]
    RMSE[feature_set] = {}
    for _model in [GP, TPOT]:
    # Pull out information/initialize
        model = _model[feature_set]['model']
        model_name = _model[feature_set]['model_name']
        RMSE[feature_set][model_name] = dict.fromkeys(ADS)
        traces = []
        
        # Create a parity plot where each adsorbate is shown. We do that by pulling out
        # data for each adsorbate and then plotting them.
        for ads in ADS:
            # We loop through all of our data and pull out the transformed features (x),
            # the DFT energy (y), and the user-readable features (text).
            x = []
            y = []
            text = []
            for i, _ads in enumerate(data['adsorbate']):
                if _ads == ads:
                    x.append(X[feature_set][i])
                    y.append(Y[feature_set][i])
                    if feature_set == 'energy_fr_coordcount':
                        text.append('Site:  %s' \
                                    % (data['coordination'][i]))
                    elif feature_set == 'energy_fr_coordcount_ads':
                        text.append('Site:  %s' \
                                    % (data['coordination'][i]))
                    elif feature_set == 'energy_fr_coordcount_nncoord_ads':
                        text.append('Site:  %s\rNNNeighbor:  %s' \
                                    % (data['coordination'][i],
                                       data['nextnearestcoordination'][i]))
                    elif feature_set == 'energy_fr_gcn_ads':
                        text.append('Site:  %s' \
                                    % (data['coordination'][i]))
                    else:
                        raise Exception('You still need to hard-code the text for the %s' \
                                        % feature_set)
            # Use the transformed features (x) to calculate a predicted energy (y_predicted).
            # Then add it to `traces` for plotting. We use EAFP to use either a blocked
            # model or the whole model
            try:
                y_predicted = model[ads].predict(np.array(x))
            except TypeError:
                y_predicted = model.predict(np.array(x))
            traces.append(go.Scatter(x=y_predicted,
                                     y=y,
                                     mode='markers',
                                     text=text,
                                     name=ads))
            
            # Calculate the RMSE for this particular model
            RMSE[feature_set][model_name][ads] = \
                    math.sqrt(metrics.mean_squared_error(y, y_predicted))
        # Calculate the RMSE for the whole model
        rmses = RMSE[feature_set][model_name]
        RMSE[feature_set][model_name]['total'] = \
            math.sqrt(sum([rmse**2 for i, rmse in rmses.iteritems()])/len(rmses))
            
        # Create a diagonal line for the parity plot
        lims = [-4, 6]
        traces.append(go.Scatter(x=lims, y=lims,
                                 line=dict(color=('black'), dash='dash'), name='Parity line'))
        # Format and plot
        layout = go.Layout(xaxis=dict(title='Regressed (eV)'),
                           yaxis=dict(title='DFT (eV)'),
                           title='Predicting %s using a %s model; RMSE = %0.3f eV' \
                                 % (feature_set,
                                    model_name,
                                    RMSE[feature_set][model_name]['total']))
        iplot(go.Figure(data=traces, layout=layout))

###### Alamo

In [ ]:
# Create Plotly plots for each dictionary-type model
for feature_set in FEATURE_SETS:
    for model in [ALA[feature_set]]:
        traces = []
        # Create a parity plot where each adsorbate is shown. We do that by pulling out
        # data for each adsorbate and then plotting them.
        for ads in np.unique(DATA[feature_set]['adsorbate']):
            # We loop through all of our data and pull out the vectorized coordination (x),
            # the DFT energy (y), and the coordination site (text).
            x = []
            y = []
            text = []
            for i, _ads in enumerate(DATA[feature_set]['adsorbate']):
                if _ads == ads:
                    x.append(X[i][feature_set])
                    y.append(Y[i][feature_set])
                    if feature_set == 'energy_fr_coordcount_ads':
                        text.append('Site:  %s' \
                                    % DATA[feature_set]['coordination'][i])
                    elif feature_set == 'energy_fr_coordcount_neighborcount_ads':
                        text.append('Site:  %s\rNeighbor:  %s' \
                                    % (DATA[feature_set]['coordination'][i],
                                       DATA[feature_set]['nextnearestcoordination'][i]))
                    else:
                        raise Exception('You still need to hard-code the text for the %s' \
                                        % feature_set)

            # Do some footwork because Alamo returns a lambda function that doesn't accept np arrays
            def model_predict(factors):
                '''
                Turn a vector of input data, `factors`, into the model's guessed output. We use
                this function to do so because lambda functions suck. We should address this by
                making alamopy output a better lambda function.
                '''
                args = dict.fromkeys(range(0, len(factors)-1), None)
                for j, factor in enumerate(factors):
                    args[j] = factor
                return model['f(model)'](args[0], args[1], args[2], args[3], args[4], args[5], args[6], args[7], args[8], args[9], args[10], args[11], args[12], args[13], args[14], args[15], args[16], args[17], args[18], args[19], args[20], args[21], args[22], args[23], args[24], args[25])
            y_predicted = map(model_predict, x)

            # Plot
            traces.append(go.Scatter(x=y_predicted,
                                     y=y,
                                     mode='markers',
                                     text=text,
                                     name=ads))
        # Create a diagonal line for the parity plot
        lims = [-4, 6]
        traces.append(go.Scatter(x=lims, y=lims,
                                 line=dict(color=('black'), dash='dash'), name='Parity line'))
        # Format and plot
        layout = go.Layout(xaxis=dict(title='Regressed (eV)'),
                           yaxis=dict(title='DFT (eV)'),
                           title='Predicting %s using a %s model; RMSE = %0.3f eV' \
                                 % (feature_set,
                                    model['name'],
                                    math.sqrt(metrics.mean_squared_error(Y_TEST, map(model_predict, X_TEST)))))
        iplot(go.Figure(data=traces, layout=layout))